In [134]:
import numpy as np
from numpy.random import choice
from numpy.random import multinomial
from numpy.random import dirichlet
import pandas as pd
from lda import *
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.preprocessing import normalize
from math import fsum
import time

In [2]:
# data

dtm = datasets.load_reuters()
vocab = datasets.load_reuters_vocab()
titles = datasets.load_reuters_titles()

In [3]:
# utils

class DocTermMatrix(object):
    def __init__(self, corpus, **kwargs):
        '''
        corpus: a collection of raw documents
        with **kwargs you can pass arguments of CountVectorizer
        creates a doc-term matrix instance (as scipy sparse matrix)
        '''

        vectorizer = CountVectorizer(**kwargs)
        self.num_docs = len(corpus)
        self.vocab_size = len(vectorizer.vocabulary_)
        self.shape = (self.num_docs, self.vocab_size)
        self.matrix = vectorizer.fit_transform(docs)

    def dataframe(self):
        '''
        returns the doc-term matrix as pandas dataframe
        '''
        return pd.DataFrame(x1.toarray(), columns = vectorizer.get_feature_names())

    def length(self):
        """
        Calculates the number of words in each document, i.e. its length.
        Returns an array of the same size as the corpus.
        """
        # lengths_list = np.zeros(self.row_dim)
        lengths_list = self.df().apply(lambda x: len(x.nonzero()[0]), axis=1)
        return lengths_list

    @classmethod
    def word_index(cls, doc_row):
        """
        doc_row: document vector (of size V, vocab-size)
        Returns an array of repeated indices/terms for the words in the document;
        the indices/terms are repeated the same number of times as the number of occurrences for the corresponding word;
        the length of the array is thus equal to the document length.
        """
        # doc_row = np.array(doc_row)
        for idx in doc_row.nonzero()[0]:
            for i in range(int(doc_row[idx])):
                yield idx

# My journey through hLDA:

Below, firstly, is my implementation of the hierarchical prior for the collapsed Gibbs sampler, largely following the notation of the notes.

I have my questions in the code, but here is an overarching one:
    - I can understand how, by implementing the Polya urn scheme with probabilities that reward previously drawn topics ("rich get richer"?), we would get a few topics being general and others more specific. It is not obvious to me where we made sure that there is a single, unique topic at the root of every path through the hierarchy. [which is how it is plotted in Blei et al. 2004 - 'Hierarchical Topic Models and the Nested Chinese Restaurant Process']

In [135]:
class hLDA(object):
    def __init__(self, tree_depth, alpha, gamma, eta):
        '''
        gamma: the parameter for the (base) Dirichlet from which the prior vector ('m') is drawn.
                in the notes' notation, this is alpha'.
        tree_depth: a parameter for the number of levels in a hierarchy.
        '''
        self.gamma = gamma
        self.alpha = alpha
        self.k = tree_depth
        self.eta = eta
        

    def priors(self, dtm, seed=None):
        self.num_docs, self.vocab_size = dtm.shape

        # number of times topic K allocaition variables generate term/word V
        self.nkv = np.zeros((self.k, self.vocab_size))

        #### dtm = DocTermMatrix(corpus).dataframe()
        # Document-specific vectors that record the multiplicity of each label
        self.local_count = np.zeros((self.num_docs, self.k))
        # Global vector which keeps tracks of all draws from the base measure
        self.global_count = np.zeros(self.k)

        if seed is not None:
            np.random.seed(seed)

        # Simulating from the hierarchical prior for the first document
        sample = {}
        sample[0] = multinomial(1,[1/self.k]*self.k).nonzero()[0][0] # sample s1
        R = set(sample.values())
        M = 1
        z = {}
        # set the topic of the first word in the first doc to s1
        z[(0,0)] = sample[0]
        self.global_count[sample[0]] += 1
        self.local_count[0, sample[0]] += 1
        for n, w in enumerate(DocTermMatrix.word_index(dtm[0,:])): # skip the word sampled first above?
            draw = self.roll_die_first_doc(n_d=self.local_count, n_g=self.global_count, N=n, M=M)
            if draw is not None and draw < self.k:
                z[(0,n+1)] = draw
                self.local_count[0, draw] += 1
                #M += 1 #
            elif draw >= self.k and draw < 2*self.k:
                sample[M] = draw - self.k # assign the s_{Mn+1} = s

                # update local and global count?
                self.local_count[0, sample[M]] += 1
                self.global_count[sample[M]] += 1
                z[(0, n+1)] = sample[M]
                M += 1
                # set z_{n+1} equal to?
                
            elif draw >= 2*self.k:
                new_s = multinomial(1,[1/self.k]*self.k).nonzero()[0][0]
                sample[M] = new_s
                M += 1
                self.global_count[new_s] += 1
                self.local_count[0, new_s] += 1
                # set z_{n+1} equal to?
                z[(0, n+1)] = new_s
            R = set(sample)

        # hierarchical prior across documnents
        # dtm = pd.DataFrame(dtm)
        # for idx, bow in dtm[1:].iterrows()
        # then idx+1 needs to be changed to idx
        for idx, bow in enumerate(dtm[1:]): # bow = bag-of-words
            for n, w in enumerate(DocTermMatrix.word_index(bow)):
                if n == 0:
                    draw = self.roll_die_first_word(n_d=self.local_count, n_g=self.global_count, M=M, doc_idx = idx+1)
                    if draw < self.k:
                        z[(idx+1,n)] = draw # double-check
                        self.local_count[idx+1, draw] = 1
                        #M += 1
        # do we increment counts for M_{k,v} here too? how does this affect the prior of beta vs. usual case?
                        self.nkv[z[(idx+1,n)],w] += 1
                    else:
                        sample[M] = multinomial(1,[1/self.k]*self.k).nonzero()[0][0]
                        z[(idx+1,n)] = sample[M] # is this the s in the notes?
                        self.local_count[idx+1, sample[M]] = 1
                        self.global_count[sample[M]] += 1
                        M += 1
                        self.nkv[z[(idx+1,n)],w] += 1
                else:
                    draw = self.roll_die_nth_word(n_d=self.local_count, n_g=self.global_count, N=n, M=M, doc_idx=idx+1)
                    if draw < self.k:
                        self.local_count[idx+1, draw] += 1
                        z[(idx+1, n)] = draw
                        self.nkv[z[(idx+1,n)],w] += 1

                    else:
                        sample[M] = multinomial(1,[1/self.k]*self.k).nonzero()[0][0]
                        z[(idx+1,n)] = sample[M]
                        self.global_count[sample[M]] += 1
                        self.local_count[idx+1, sample[M]] += 1
                        M += 1
                        self.nkv[z[(idx+1,n)],w] += 1
        return M, z

    def collapsed_gibbs(self, dtm, maxiter=100, verbose=False):
        '''
        dtm: document-term matrix of shape [n_docs, n_vocab]
        '''
        M, z = self.priors(dtm)
        for iterr in range(maxiter):
            for d in range(self.num_docs):
                for i, w in enumerate(DocTermMatrix.word_index(dtm[d,:])):
                    self.update_count(True, z, d, i, w)
                    z[(d,i)] = self.sample_topic_assignment(d, w, M)
                    self.update_count(False, z, d, i, w)
                if verbose: print('Document '+str(d) + ' complete')
            if verbose:
                print('Iteration '+str(iterr) + ' complete')
        # can pick a sample that maximises joint probability
        # and recover estimates of theta and beta from there

        return z


    def sample_topic_assignment(self, d, w, M):
        zdi_probs = np.zeros(self.k)
        #epsilon = 10e-5
        for k in range(self.k):
            prior = 1/(self.alpha + fsum(self.local_count[d, :]) - 1)
            prior = prior * (self.local_count[d,k] + self.alpha*(self.global_count[k] + self.gamma/self.k)/(self.gamma+M))
            likelihood = self.nkv[k,w] + (self.eta/self.vocab_size)
            likelihood = likelihood / (fsum(self.nkv[k,:])+self.eta)
            zdi_probs[k] = prior*likelihood
        
        zdi_probs /= fsum(zdi_probs)
        #zdi_probs = zdi_probs[:-1]
        #if sum(zdi_probs)>1.0:
            #zdi_probs -= epsilon
        zdi = multinomial(1, zdi_probs).nonzero()[0][0]
        return zdi

    def update_count(self, decrease_count, z, d, i, w):
        """
        decrease_count: Boolean
        d: 		 		document number
        i:		 		ith word [unordered] in document d
        w:		 		term w in document d

        Updates count variables to run collapsed sampling equation for LDA.
        """
        # We no longer decrement counts of N_{d,k}?
        if decrease_count: # this didn't use to happen for vanilla LDA: why?
            if self.local_count[d,z[(d,i)]] > 0:
                self.local_count[d,z[(d,i)]] -= 1
            if self.global_count[z[(d,i)]] > 0:
                self.global_count[z[(d,i)]] -= 1
            if self.nkv[z[(d,i)],w]>0:
                self.nkv[z[(d,i)],w] -= 1 # if count was zero, we don't decrement further?
        elif not decrease_count:
            self.local_count[d,z[(d,i)]] += 1
            self.global_count[z[(d,i)]] += 1
            self.nkv[z[(d,i)],w] += 1

    def roll_die_first_doc(self, n_d, n_g, N, M):
        # probabilities of picking one of s
        probs_old_draws = n_d[0, :] / (self.alpha + N+1) # N+1 due to zero-indexing

        # probabilities of a new draw being set to s
        probs_new_draw_s = n_g[:] / (self.gamma + M)
        probs_new_draw_s *= (self.alpha / (self.alpha + N+1))

        # probability of a new draw from the base measure
        prob_new_from_base = (self.alpha / (self.alpha + N+1)) * (self.gamma / (self.gamma + M))
        
        probabilities = list(probs_old_draws) + list(probs_new_draw_s) + list([prob_new_from_base])
        draw = choice(len(probabilities), p=probabilities)

        return draw

    def roll_die_first_word(self, n_d, n_g, M, doc_idx):
        # probabilities of picking one of s
        probs_old_draws = n_g[:] / (self.gamma + M)

        # probability of a new draw
        prob_new_draw = self.gamma / (M + self.gamma)

        probabilities = list(probs_old_draws) + list([prob_new_draw])
        draw = choice(len(probabilities), p=probabilities)
        return draw

    def roll_die_nth_word(self, n_d, n_g, N, M, doc_idx):
        # probabilities of picking one of s
        probs_old_draws = n_d[doc_idx, :] / (self.alpha + N)

        # probability of a new draw
        prob_new_draw = self.alpha / (self.alpha + N)

        probabilities = list(probs_old_draws) + list([prob_new_draw])
        draw = choice(len(probabilities), p=probabilities)
        return draw
    
    def _initialize(self, dtm):
        """
        dtm: a DxV document-term matrix
        Initialises the count variables N_dk, N_kv, N_k, N_d.
            K: topic
            D: document
            V: term
        """

        self.num_docs, self.vocab_size = dtm.shape

        # number of terms/words in document D that have topic allocation K
        self.ndk = np.zeros((self.num_docs, self.k))
        # number of times topic K allocaition variables generate term/word V
        self.nkv = np.zeros((self.k, self.vocab_size))
        # number of terms/words generated by topic K
        self.nk = np.zeros(self.k)

        # The Dirichlet prior for the document-topic distribution
        
        self.m = dirichlet(self.gamma * np.ones(self.k)) # m is corpus-wide
        theta = dirichlet(self.alpha * self.m, size = self.num_docs)

        # The Dirichlet prior for the topic-term distribution
        beta = dirichlet(self.eta * np.ones(self.vocab_size), size = self.k)

        # Initialize the topic assignment variables as a dictionary
            # key corresponds to (d,n): nth word in document d
            # value = {0,...,K-1} is the topic assigned to that word
        self.z = {}
        for d in range(self.num_docs):
            for i, w in enumerate(DocTermMatrix.word_index(dtm[d,:])):
                # total iterations of i will equal the document length
                # the set of w includes all terms in document d
                self.z[(d,i)] = multinomial(1, theta[d]).nonzero()[0][0]
                self.ndk[d,self.z[(d,i)]] += 1
                self.nkv[self.z[(d,i)],w] += 1
                self.nk[self.z[(d,i)]] += 1

    def run_Gibbs(self, dtm, maxIter=100):
        """
        Uncollapsed Gibbs sampling.

        matrix: the document-term matrix.
        maxIter: the number of iterations to run.

        One could construct predictive distributions for theta and beta from the posterior samples (like we do in the collapsed Gibbs).
        """
        self._initialize(dtm)
        theta_posterior = np.empty((self.num_docs, self.k)) 	# DxK matrix
        beta_posterior = np.empty((self.k, self.vocab_size)) 		# KxV matrix
        theta_samples = []
        beta_samples = []

        for iterr in range(maxIter):
            for k in range(self.k):
                # Posterior for BETA
                beta_posterior[k,:] = dirichlet(self.eta * np.ones(self.vocab_size) + self.nkv[k,:])
            for d in range(self.num_docs):
                # Posterior for THETA
                theta_posterior[d,:] = dirichlet(self.alpha * self.m + self.ndk[d,:])
            for d in range(self.num_docs):
                for i, w in enumerate(DocTermMatrix.word_index(dtm[d,:])):
                    self.update_counts(True, d, i, w)
                    self.z[(d,i)] = self.sample_topic(d, w, theta_posterior, beta_posterior)
                    # update counts
                    self.update_counts(False, d, i, w)
            # Burn-in and thinning interval set to 10 iterations
            if (iterr + 1)%10 == 0:
                theta_samples.append(theta_posterior)
                beta_samples.append(beta_posterior)
            if iterr == maxIter:
                theta_samples.append(theta_posterior)
                beta_samples.append(beta_posterior)
        
        #return z, theta_samples, beta_samples, self.ndk, self.nkv, self.nk 
        return theta_samples, beta_samples
    
    def sample_topic(self, d, w, theta, beta):
        zdi_probs = np.zeros(self.k)
        for k in range(self.k):
            zdi_probs[k] = theta[d,k]*beta[k,w] / np.dot(theta[d,:],beta[:,w])
        zdi_probs /= sum(zdi_probs)
        zdi = multinomial(1, zdi_probs).nonzero()[0][0]
        return zdi
    
    def update_counts(self, decrease_count, d, i, w):
        """
        decrease_count: Boolean
        d: 		 		document number
        i:		 		ith word [unordered] in document d
        w:		 		term w in document d

        Updates count variables to run collapsed sampling equation for LDA.
        """
        if decrease_count:
            self.ndk[d,self.z[(d,i)]] -= 1
            self.nkv[self.z[(d,i)],w] -= 1
        else:
            self.ndk[d,self.z[(d,i)]] += 1
            self.nkv[self.z[(d,i)],w] += 1

In [136]:
model = hLDA(tree_depth = 20, alpha = 0.1, gamma = 1, eta = 0.01)

In [137]:
start=time.time()
z = model.collapsed_gibbs(dtm, maxiter=50, verbose=True)
end = time.time()
total = end-start
print(total)

Document 0 complete
Document 1 complete
Document 2 complete
Document 3 complete
Document 4 complete
Document 5 complete
Document 6 complete
Document 7 complete
Document 8 complete
Document 9 complete
Document 10 complete
Document 11 complete
Document 12 complete
Document 13 complete
Document 14 complete
Document 15 complete
Document 16 complete
Document 17 complete
Document 18 complete
Document 19 complete
Document 20 complete
Document 21 complete
Document 22 complete
Document 23 complete
Document 24 complete
Document 25 complete
Document 26 complete
Document 27 complete
Document 28 complete
Document 29 complete
Document 30 complete
Document 31 complete
Document 32 complete
Document 33 complete
Document 34 complete
Document 35 complete
Document 36 complete
Document 37 complete
Document 38 complete
Document 39 complete
Document 40 complete
Document 41 complete
Document 42 complete
Document 43 complete
Document 44 complete
Document 45 complete
Document 46 complete
Document 47 complete
Do

In [152]:
print('%f hours' %(total/60 /60))

11.406035 hours


In [140]:
def compute_theta(model):
        """
        Approximate the predictive document-topic distribution THETA; vectorized implementation.
        """
        numerator = model.local_count + model.alpha 						# DxK matrix
        denominator = np.sum(numerator, axis=1)[:, np.newaxis] # Dx1 vector: total sum of elements for each row of numerator

        # Divide each element in row d of numerator by element d of denominator
        theta = numerator / denominator
        return theta

def compute_beta(model):
        """
        Approximate the predictive topic-term distributions BETA; vectorized implementation.
        """
        numerator = model.nkv + model.eta 							# KxV matrix
        denominator = np.sum(numerator, axis=1)[:, np.newaxis] # Kx1 vector: total sum of elements for each row of numerator

        beta = numerator / denominator
        return beta


In [141]:
doc_topic = compute_theta(model)

In [142]:
topic_word = compute_beta(model)

In [143]:
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: miami versace cunanan police beach gay thursday home
Topic 1: quebec polish cardinal italian mrs five days aides
Topic 2: clinton bishop son wright funeral late catholic father
Topic 3: king church national bishops runcie media post extrapyramidal
Topic 4: mother teresa order heart charity calcutta nuns missionaries
Topic 5: world million people first elvis life show own
Topic 6: royal camilla king marriage romania michael family couple
Topic 7: church years told last against year public former
Topic 8: pope vatican surgery operation paul doctors rome john
Topic 9: east prize peace timor belo indonesia nobel award
Topic 10: british war family died old london ceremony party
Topic 11: election order official successor west members year 10
Topic 12: sale opera estate leigh british deal cassisa taylor
Topic 13: papers ramos gandhi hwang phillips forbes indian fbi
Topic 14: u.s harriman president france paris minister political germany
Topic 15: simpson russian nirmala russia tsar 

### uncollapsed sampler

In [124]:
model_1 = hLDA(tree_depth = 20, alpha = 0.1, gamma = 0.1, eta = 0.01)

In [126]:
result = model_1.run_Gibbs(dtm)

In [127]:
result[0][-1].shape

(395, 20)

In [128]:
result[1][-1].shape   

(20, 4258)

In [129]:
doc_topic = result[0][-1]
topic_word = result[1][-1]

In [130]:
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: church years king prince royal last charles first
Topic 1: lung covering dynasty while magazines fur maybe dennis
Topic 2: ellen gays gay film marsalis ran 100,000 tipped
Topic 3: entitled respected connecticut wojtyla ambassador staff polish part
Topic 4: city salonika clinton byzantine modern told fringe athos
Topic 5: stalin russian russia moscow yeltsin radzinsky kremlin georgy
Topic 6: order nobel indonesia teresa east belo mother nuns
Topic 7: world weight dream balmoral opening exercising heartbeat gorbachev
Topic 8: offices big house humour notably fatima mixed set
Topic 9: brother harry landscape balabagan leaves problems position bbc
Topic 10: charles harriman u.s camilla mother diana parker marriage
Topic 11: mistake spokesman india letters ghandi packed writings century
Topic 12: pope mother teresa vatican hospital harriman church john
Topic 13: germany jews film last people german scientology flynt
Topic 14: nuclear vanunu film israeli intellectual critics directo

### what the vanilla LDA package finds

In [132]:
model_package = lda.LDA(n_topics=20, n_iter=1500, random_state=1)
model_package.fit(dtm)  # model.fit_transform(X) is also available
topic_word = model_package.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: british churchill sale million major letters west britain
Topic 1: church government political country state people party against
Topic 2: elvis king fans presley life concert young death
Topic 3: yeltsin russian russia president kremlin moscow michael operation
Topic 4: pope vatican paul john surgery hospital pontiff rome
Topic 5: family funeral police miami versace cunanan city service
Topic 6: simpson former years court president wife south church
Topic 7: order mother successor election nuns church nirmala head
Topic 8: charles prince diana royal king queen parker bowles
Topic 9: film french france against bardot paris poster animal
Topic 10: germany german war nazi letter christian book jews
Topic 11: east peace prize award timor quebec belo leader
Topic 12: n't life show told very love television father
Topic 13: years year time last church world people say
Topic 14: mother teresa heart calcutta charity nun hospital missionaries
Topic 15: city salonika capital buddhist c